In [22]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

from numpy.polynomial.polynomial import polyfit

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.api as sm
import statsmodels
#import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

import warnings
warnings.filterwarnings("ignore")

### First Steps

Here we are going to combine the trade and autos data set...

In [23]:
fig_path = "C:\\github\\expenditure_tradeshocks\\figures"

In [24]:
file_path = os.getcwd()
#file_path = "F:\\POLK AUTO DATA\\"

trade_county = pq.read_table(file_path + "\\data\\trade_employment_all.parquet").to_pandas()

In [25]:
trade_county["time"] = pd.to_datetime(trade_county.time)

trade_county.set_index(["area_fips", "time"],inplace = True)

In [26]:
trade_county.head(30)

total_exp_pc  china_exp_pc    tariff  emplvl_2017  \
area_fips time                                                            
10001     2017-01-01    522.580597     58.368700  1.068759       2843.0   
          2017-02-01    502.309185     48.422056  1.068802       2843.0   
          2017-03-01    571.161328     42.286424  1.068895       2843.0   
          2017-04-01    509.278875     33.702476  1.068895       2843.0   
          2017-05-01    519.176693     33.878036  1.068895       2843.0   
          2017-06-01    509.383926     30.102481  1.068895       2843.0   
          2017-07-01    492.508803     32.581600  1.068895       2843.0   
          2017-08-01    509.577144     35.329198  1.068895       2843.0   
          2017-09-01    495.123610     44.380139  1.068895       2843.0   
          2017-10-01    569.787487     73.830992  1.068895       2843.0   
          2017-11-01    571.520102     73.114154  1.068895       2843.0   
          2017-12-01    553.649765     59.845960  1.068895       2843.0   
          2018-01-01    516.700915     48.624283  1.068805       2843.0   
          2018-02-01    516.145992     44.607737  1.068897       2843.0   
          2018-03-01    587.864923     42.979806  1.068894       2843.0   
          2018-04-01    559.410137     36.213274  1.393797       2843.0   
          2018-05-01    579.008640     33.254994  1.393798       2843.0   
          2018-06-01    558.919118     28.428818  1.393798       2843.0   
          2018-07-01    524.597360     25.288644  2.221774       2843.0   
          2018-08-01    545.616139     25.092559  2.221774       2843.0   
          2018-09-01    511.239593     21.133593  2.235515       2843.0   
          2018-10-01    562.324040     24.854552  2.485311       2843.0   
          2018-11-01    555.172968     25.784350  2.426361       2843.0   
          2018-12-01    524.630933     23.745690  2.426361       2843.0   
          2019-01-01    527.484006     29.783300  2.424109       2843.0   
          2019-02-01    505.126798     33.942512  2.424092       2843.0   
          2019-03-01    573.829215     35.695393  2.424058       2843.0   
          2019-04-01    541.332336     27.687231  2.424058       2843.0   
          2019-05-01    553.302909     33.524619  2.424057       2843.0   
          2019-06-01    526.449004     37.436249  2.424056       2843.0   

                       fips  total_employment  emply_month  
area_fips time                                              
10001     2017-01-01  10001           29514.0          0.0  
          2017-02-01  10001           29514.0          0.0  
          2017-03-01  10001           29514.0          0.0  
          2017-04-01  10001           29514.0          0.0  
          2017-05-01  10001           29514.0          0.0  
          2017-06-01  10001           29514.0          0.0  
          2017-07-01  10001           29514.0          0.0  
          2017-08-01  10001           29514.0          0.0  
          2017-09-01  10001           29514.0          0.0  
          2017-10-01  10001           29514.0          0.0  
          2017-11-01  10001           29514.0          0.0  
          2017-12-01  10001           29514.0          0.0  
          2018-01-01  10001           29514.0          0.0  
          2018-02-01  10001           29514.0          0.0  
          2018-03-01  10001           29514.0          0.0  
          2018-04-01  10001           29514.0          0.0  
          2018-05-01  10001           29514.0          0.0  
          2018-06-01  10001           29514.0          0.0  
          2018-07-01  10001           29514.0          0.0  
          2018-08-01  10001           29514.0          0.0  
          2018-09-01  10001           29514.0          0.0  
          2018-10-01  10001           29514.0          0.0  
          2018-11-01  10001           29514.0          0.0  
          2018-12-01  10001           29514.0          0.0  
          2019-01-01  10001        

In [27]:
trade_county["log_tariff"] = np.log(1+.01*trade_county["tariff"])

trade_county["log_exp_total"] = np.log(trade_county["total_exp_pc"]).replace(-np.inf, np.nan)

trade_county["log_exp_china"] = np.log(trade_county["china_exp_pc"]).replace(-np.inf, np.nan)

trade_county["log_employment"] = np.log(trade_county["emply_month"]).replace(-np.inf, np.nan)

trade_county["const"] = 1

In [28]:
trade_county.reset_index(inplace = True)

trade_county.rename({"area_fips": "GEOFIPS"},axis = 1, inplace = True)

trade_county["state_fips"] = trade_county["GEOFIPS"].astype(str).str[0:2]

trade_county["GEOFIPS"] = trade_county["GEOFIPS"].astype(int)

In [29]:
#trade_county["log_value"] = np.log(trade_county["value"]).replace(-np.inf,np.nan)

trade_county.set_index(["GEOFIPS", "time"], inplace = True)

In [30]:
trade_county.head()

total_exp_pc  china_exp_pc    tariff  emplvl_2017   fips  \
GEOFIPS time                                                                   
10001   2017-01-01    522.580597     58.368700  1.068759       2843.0  10001   
        2017-02-01    502.309185     48.422056  1.068802       2843.0  10001   
        2017-03-01    571.161328     42.286424  1.068895       2843.0  10001   
        2017-04-01    509.278875     33.702476  1.068895       2843.0  10001   
        2017-05-01    519.176693     33.878036  1.068895       2843.0  10001   

                    total_employment  emply_month  log_tariff  log_exp_total  \
GEOFIPS time                                                                   
10001   2017-01-01           29514.0          0.0    0.010631       6.258779   
        2017-02-01           29514.0          0.0    0.010631       6.219216   
        2017-03-01           29514.0          0.0    0.010632       6.347672   
        2017-04-01           29514.0          0.0    0.010632       6.232996   
        2017-05-01           29514.0          0.0    0.010632       6.252244   

                    log_exp_china  log_employment  const state_fips  
GEOFIPS time                                                         
10001   2017-01-01       4.066780             NaN      1         10  
        2017-02-01       3.879955             NaN      1         10  
        2017-03-01       3.744466             NaN      1         10  
        2017-04-01       3.517571             NaN      1         10  
        2017-05-01       3.522767             NaN      1         10

### Summary Statistics

### Simple Cuts of the Auto Data by Tariff Exposure

Here is the strategy. First, compute the quantiles of tariff exposure for each county. Then compute year over year sales growth for each county. Note that since this is in growth rates, this takes out any county level "fixed effect" and removes any seasonal component. 

Then we will simply plot year over year sales growth, for the (mean or median) county in a particular part of the distribution of tariff exposure. For example, compare the sales growth for the counties which are the most exposed and then compare then to the counties which are the least exposed.

In [31]:
# First take growth rates

# note , I'm a bit confused about why the sorting,
# here is that because of some missing values, the resulting dateframe from the 
# first operation may be out of place, so we need to resort things to make sure that
# the time difference is correct.

trade_county["tariff_change"] = trade_county.groupby(["GEOFIPS"]).tariff.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

trade_county["log_tariff_change"] = trade_county.groupby(["GEOFIPS"]).log_tariff.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

trade_county["total_trade_growth"] = trade_county.groupby(["GEOFIPS"]).log_exp_total.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

trade_county["china_trade_growth"] = trade_county.groupby(["GEOFIPS"]).log_exp_china.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

### Regression Analysis


In [32]:
trade_county.columns

Index(['total_exp_pc', 'china_exp_pc', 'tariff', 'emplvl_2017', 'fips',
       'total_employment', 'emply_month', 'log_tariff', 'log_exp_total',
       'log_exp_china', 'log_employment', 'const', 'state_fips',
       'tariff_change', 'log_tariff_change', 'total_trade_growth',
       'china_trade_growth'],
      dtype='object')

#### Total Trade, Weighted

In [33]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','total_employment']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['total_employment'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.1130
Estimator:                   PanelOLS   R-squared (Between):             -0.0189
No. Observations:               53226   R-squared (Within):               0.2180
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.1130
Time:                        11:10:06   Log-likelihood                 5.902e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   2.088e+04
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,53224)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             1369.5
                            

In [34]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','total_employment']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['total_employment'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights,
               time_effects = True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0186
Estimator:                   PanelOLS   R-squared (Between):              0.0199
No. Observations:               53226   R-squared (Within):               0.1108
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0705
Time:                        11:10:06   Log-likelihood                 6.794e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   2.072e+04
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,53207)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             558.51
                            

In [35]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','total_employment']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['total_employment'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights, entity_effects=True,
               time_effects = True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0162
Estimator:                   PanelOLS   R-squared (Between):              0.0192
No. Observations:               53226   R-squared (Within):               0.0976
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0628
Time:                        11:10:07   Log-likelihood                 9.867e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   3.776e+04
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,50251)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             227.78
                            

#### Now do a check on pretrend issues...

In [36]:
trans = lambda df: df.iloc[26]

#trade_county.groupby(["GEOFIPS"]).log_tariff.transform(trans)

trade_county["lead_tariff"] = trade_county.groupby(["GEOFIPS"]).log_tariff_change.transform(trans)

In [37]:
################################################
name = "after"
date = "2018-6-1"

event_study = pd.merge(trade_county.reset_index(), 
                       pd.get_dummies(trade_county.index.get_level_values(1) > date)[1].rename(name),
                left_index = True, right_index= True)

event_study[name] = event_study.lead_tariff.multiply(event_study[name], axis = 0)

event_study.set_index(["GEOFIPS", "time"],inplace = True)

In [38]:
all_vars = ["const", "log_tariff_change",'total_employment','total_trade_growth',
           "lead_tariff", "after"]

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = event_study[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const", "lead_tariff", "after" ]

weights = dataset["total_employment"].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

print(weights.shape)
print(dataset.total_trade_growth.shape)

mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights, time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

(58536, 1)
(58536,)
                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0152
Estimator:                   PanelOLS   R-squared (Between):              0.0166
No. Observations:               53226   R-squared (Within):               0.0743
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0487
Time:                        11:10:09   Log-likelihood                 6.785e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.023e+04
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(2,53206)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             103.52
        

Same story as with employment and autos here.  Slightly negative, but not distiuishable from zero on the common component. Larger negative value on the differential effect after implementation of the tariff. Should also push this back to 2016.

#### Total Trade, unweighted

In [39]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth']

dataset = trade_county[all_vars].dropna()

exog_vars = ["const","log_tariff_change"]

mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars])

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0442
Estimator:                   PanelOLS   R-squared (Between):             -0.0245
No. Observations:               56183   R-squared (Within):               0.0941
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0442
Time:                        11:10:09   Log-likelihood                 1.735e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2596.9
Entities:                        3252   P-value                           0.0000
Avg Obs:                       17.276   Distribution:                 F(1,56181)
Min Obs:                       0.0000                                           
Max Obs:                       19.000   F-statistic (robust):             1650.6
                            

In [40]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth']

dataset = trade_county[all_vars].dropna()

exog_vars = ["const","log_tariff_change"]

mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], time_effects = True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0037
Estimator:                   PanelOLS   R-squared (Between):              0.0037
No. Observations:               56183   R-squared (Within):               0.0346
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0216
Time:                        11:10:09   Log-likelihood                 2.307e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      210.53
Entities:                        3252   P-value                           0.0000
Avg Obs:                       17.276   Distribution:                 F(1,56163)
Min Obs:                       0.0000                                           
Max Obs:                       19.000   F-statistic (robust):             240.71
                            

In [41]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth']

dataset = trade_county[all_vars].dropna()

exog_vars = ["const","log_tariff_change"]

mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars],time_effects = True,entity_effects=True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0039
Estimator:                   PanelOLS   R-squared (Between):              0.0035
No. Observations:               56183   R-squared (Within):               0.0379
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0234
Time:                        11:10:10   Log-likelihood                 4.475e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      206.37
Entities:                        3252   P-value                           0.0000
Avg Obs:                       17.276   Distribution:                 F(1,53207)
Min Obs:                       0.0000                                           
Max Obs:                       19.000   F-statistic (robust):             213.99
                            

#### Chine Trade, Weighted

In [42]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','total_employment']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['total_employment'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.1643
Estimator:                   PanelOLS   R-squared (Between):              0.1438
No. Observations:               53226   R-squared (Within):               0.1696
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.1643
Time:                        11:10:11   Log-likelihood                -1.356e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8836.9
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,53224)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             1278.4
                            

In [43]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','total_employment']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['total_employment'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights, time_effects = True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0537
Estimator:                   PanelOLS   R-squared (Between):              0.1726
No. Observations:               53226   R-squared (Within):               0.1272
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.1365
Time:                        11:10:11   Log-likelihood                   -9359.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1112.7
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,53207)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             357.37
                            

In [44]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','total_employment']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['total_employment'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights,
               time_effects = True, entity_effects=True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0124
Estimator:                   PanelOLS   R-squared (Between):              0.1446
No. Observations:               53226   R-squared (Within):               0.0949
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.1050
Time:                        11:10:12   Log-likelihood                   -2109.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6089.2
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,50251)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             152.30
                            

In [45]:
#### Now do a check on pretrend issues...

In [46]:
all_vars = ["const", "log_tariff_change",'total_employment','china_trade_growth',
           "lead_tariff", "after"]

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = event_study[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const", "lead_tariff", "after" ]

weights = dataset["total_employment"].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

print(weights.shape)
print(dataset.china_trade_growth.shape)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights, time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

(58536, 1)
(58536,)
                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0515
Estimator:                   PanelOLS   R-squared (Between):              0.1508
No. Observations:               53226   R-squared (Within):               0.0919
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.1039
Time:                        11:10:13   Log-likelihood                   -9420.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      494.28
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(2,53206)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             74.827
        

This is the one place where it looks like there is a pretrend. Should download the longer dataset, say back to 2016, and see what happens. A lot of this may look like anticipation effects.

#### China Trade Unweighted

In [47]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars])

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0974
Estimator:                   PanelOLS   R-squared (Between):              0.1165
No. Observations:               53226   R-squared (Within):               0.0933
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0974
Time:                        11:10:13   Log-likelihood                -5.196e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      5745.8
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,53224)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             1526.5
                            

In [48]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','emplvl_2017']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,"2018-01-01":"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars],
               time_effects = True, entity_effects=True)

fe_res = mod.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0121
Estimator:                   PanelOLS   R-squared (Between):              0.1282
No. Observations:               53226   R-squared (Within):               0.0597
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0719
Time:                        11:10:14   Log-likelihood                -3.915e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      614.09
Entities:                        2957   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                 F(1,50251)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             484.60
                            